# Sign Language Model Detector

### Regarding the Project

##### DataSet
* Same Image Size
* Similar Background
* Intensity range
* Distributed Based on letter Presentation

##### Split Data
* Single Train/Val/Test 75%/12.5%/12.5%

##### Implement Optimizers
* Improve Runtime Due to CPU usage (Mac core)

##### Metrics
* Accuracy
* Confusion Matrix
* ROC Curve
* f1 Score

##### Set your callbacks and track the experiments
* Early stopping - patience
* Model check point
* Learning rate scheduler

##### Set the main hyperparameters
* batch size
* learning rate
* number of epochs

##### Train Model
* Adam as Optimizer
* Batch Size = 64
* Using TensorFlow

##### Implement Test
* Apply Predictions
* Extract relavent metrics
* Measure inference Time

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# import image processing libraries
import cv2
import skimage
from skimage.transform import resize

# import tensorflow and keras
import tensorflow as tf
from tensorflow import keras
import os
from tensorflow.keras.utils import to_categorical
from glob import glob
from random import sample
from random import random